# 1. 环境准备

## 1.1 安装transformers

In [ ]:
pip install transformers

## 1.2 查看GPU

In [26]:
!nvidia-smi

Tue Aug  3 09:01:12 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    35W / 250W |   2463MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# 4. How to use

## 4.1 可以通过pipeline来直接使用这个模型进行掩码语言建模问题（MLM）

作者提供的英文demo

In [ ]:
from transformers import pipeline # transformers是基准，pipeline理解为一种执行器？
unmasker = pipeline('fill-mask', model='bert-base-uncased') # 任务：fill-mask，模型：bert-base-uncased
unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731059312820435,
  'sequence': "hello i'm a fashion model.",
  'token': 4827,
  'token_str': 'fashion'},
 {'score': 0.08774515986442566,
  'sequence': "hello i'm a role model.",
  'token': 2535,
  'token_str': 'role'},
 {'score': 0.05338393896818161,
  'sequence': "hello i'm a new model.",
  'token': 2047,
  'token_str': 'new'},
 {'score': 0.04667220264673233,
  'sequence': "hello i'm a super model.",
  'token': 3565,
  'token_str': 'super'},
 {'score': 0.027095947414636612,
  'sequence': "hello i'm a fine model.",
  'token': 2986,
  'token_str': 'fine'}]

自己做一个测试，看看bert-chinese能不能用于中文填充

In [ ]:
from transformers import pipeline # transformers是基准，pipeline理解为一种执行器？
unmasker = pipeline('fill-mask', model='bert-base-chinese') # 任务：fill-mask，模型：bert-base-uncased
a = unmasker("北京是[MASK]国的首都.")
print(a[0])

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'sequence': '北 京 是 中 国 的 首 都.', 'score': 0.9919257760047913, 'token': 704, 'token_str': '中'}


In [31]:
from transformers import pipeline # transformers是基准，pipeline理解为一种执行器？
unmasker = pipeline('fill-mask', model='bert-base-chinese') # 任务：fill-mask，模型：bert-base-uncased
unmasker("张伯伯，李伯伯，饽饽[MASK]里买饽饽.")

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.3059212267398834,
  'sequence': '张 伯 伯 ， 李 伯 伯 ， 饽 饽 哪 里 买 饽 饽.',
  'token': 1525,
  'token_str': '哪'},
 {'score': 0.0894886776804924,
  'sequence': '张 伯 伯 ， 李 伯 伯 ， 饽 饽 这 里 买 饽 饽.',
  'token': 6821,
  'token_str': '这'},
 {'score': 0.08113468438386917,
  'sequence': '张 伯 伯 ， 李 伯 伯 ， 饽 饽 家 里 买 饽 饽.',
  'token': 2157,
  'token_str': '家'},
 {'score': 0.06640081107616425,
  'sequence': '张 伯 伯 ， 李 伯 伯 ， 饽 饽 店 里 买 饽 饽.',
  'token': 2421,
  'token_str': '店'},
 {'score': 0.05617409572005272,
  'sequence': '张 伯 伯 ， 李 伯 伯 ， 饽 饽 那 里 买 饽 饽.',
  'token': 6929,
  'token_str': '那'}]

## 4.2 这里介绍了如何使用此模型来获取Pytorch中给定文本的特征

In [ ]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
# text = "Replace me by any text you'd like."
text = "Germany beat Argentina 2-0 in the World Cup Final."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

print(output['pooler_output'].shape) # 
print(output['last_hidden_state'].shape) # torch.Size([1, 14, 768])
print(output)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 768])
torch.Size([1, 14, 768])
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.7806, -0.5052, -0.4803,  ..., -0.5686,  0.5664,  0.0422],
         [-0.2767, -0.0094, -0.5687,  ..., -0.9396,  0.5665,  0.5992],
         [-0.6992, -0.5184,  0.0645,  ..., -0.5637,  0.3959, -0.1853],
         ...,
         [-0.0658, -0.3086,  0.4791,  ..., -0.7885,  0.2312, -0.7588],
         [ 0.5805, -0.1045, -0.4543,  ...,  0.0747, -0.4612, -0.3782],
         [ 0.7057,  0.0034, -0.4283,  ..., -0.0122, -0.6359, -0.3025]]],
       grad_fn=<NativeLayerNormBackward>), pooler_output=tensor([[-0.9522, -0.5991, -0.9351,  0.8507,  0.8070, -0.3187,  0.9368,  0.4064,
         -0.7434, -1.0000, -0.6296,  0.9396,  0.9925,  0.1530,  0.9511, -0.8412,
         -0.5568, -0.6162,  0.4094, -0.5936,  0.7155,  1.0000, -0.2538,  0.5125,
          0.4883,  0.9832, -0.7997,  0.9457,  0.9806,  0.8179, -0.5707,  0.3542,
         -0.9948, -0.1905, -0.9490, -0.9971,  0.5461, -0.8030,  0.05

## 4.3 这里介绍了如何使用此模型来获取TensorFlow中给定文本的特征

In [23]:
from transformers import BertTokenizer, TFBertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained("bert-base-uncased")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
